In [1]:
import numpy as np
import pandas as pd
# pd.set_option('display.max_columns', None)

import sklearn
from sklearn import model_selection as ms
# sklearn.set_config(print_changed_only=False)

# from sklearn.preprocessing import OneHotEncoder
# from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
# from sklearn.model_selection import GridSearchCV
# from sklearn import linear_model as lm

import cupy as cp
import cudf
import cuml
# from cuml.preprocessing import OneHotEncoder
# from cuml.preprocessing import MinMaxScaler
# from cuml import train_test_split as cutts

import dask_ml
import dask_ml.model_selection as dcv
# from dask_ml.preprocessing import DummyEncoder
# from dask_ml.preprocessing import Categorizer
# from dask_ml.preprocessing import OneHotEncoder
from dask_ml.preprocessing import MinMaxScaler

import xgboost

import copy

import itertools

import re

import import_ipynb

from _Self_Written_Functions_Sheet import (conditions_edit, nbrhd_rank, drop_nbrhd,
totallivsf_add, drop_grlivarea, bsmtfin_add, drop_bsmtfinsf12, totalporchsf_add,
drop_porches, yrsremodtosold_add, drop_yrsoldremod, fireplace_yes, drop_fireplaces,
pool_yes, drop_poolarea, totalbaths_add, drop_baths, totalbsmtbaths_add,
drop_bsmtbaths)

from _Self_Written_Functions_Sheet import dum_scale, use_og_data, comb_list_generator, rmse

importing Jupyter notebook from _Self_Written_Functions_Sheet.ipynb


In [2]:
# importing the datasets
hp_train = pd.read_csv('Proj3_Datasets/hp_2a_ranked_edited_train.csv', index_col=0)
hp_test = pd.read_csv('Proj3_Datasets/hp_2a_ranked_edited_test.csv', index_col=0)
hp_logsp = pd.read_csv('Proj3_Datasets/hp_1a_no_imputation_logsaleprice.csv',
                                index_col=0)
hp_saleprice = pd.read_csv('Proj3_Datasets/hp_1a_no_imputation_saleprice.csv')

# saving train index
hp_index = hp_train.index

combo_nonedit = pd.concat([hp_train, hp_test])
combo_nonedit.columns = hp_train.columns

# convert all possible numeric types to numeric
for col in combo_nonedit.columns: 
    combo_nonedit[col] = combo_nonedit[col].astype('float', errors='ignore')

combo_cols = list(combo_nonedit.columns)

combo = copy.deepcopy(combo_nonedit)

In [3]:
def mlr_model(x_trn, y_trn, x_tst, y_tst):
    lm = LinearRegression()
    # parameters to use
    fitintercept = [True, False]
    gparam_mlr_1 = {'fit_intercept': fitintercept}
    # setting parameters
    gs_mlr_1 = dcv.GridSearchCV(lm, gparam_mlr_1, cv=n_folds, refit=True, n_jobs=-1,
                               scoring='neg_root_mean_squared_error',
                               return_train_score=True)
    gs_mlr_1.fit(x_trn, y_trn)
    mlr_model.test_rmse = rmse(gs_mlr_1, y_tst, x_tst)    
    
def lasso_model(x_trn, y_trn, x_tst, y_tst):
    lasso = Lasso(max_iter=1000000)
    # parameters to use
    alphas = [0.0001, 0.000112, 0.000124, 0.000136, 0.000148]
    fitintercept = [True, False]
    selec = ['cyclic', 'random']
    gparam_lasso_1 = {'alpha': alphas,
                      'fit_intercept':fitintercept,
                      'selection': selec}
    # setting parameters
    gs_lasso_1 = dcv.GridSearchCV(lasso, gparam_lasso_1, cv=n_folds, refit=True, n_jobs=-1,
                                 scoring='neg_root_mean_squared_error',
                                 return_train_score=True)
    gs_lasso_1.fit(x_trn, y_trn)
    lasso_model.test_rmse = rmse(gs_lasso_1, y_tst, x_tst)

# def ridge_model(x_trn, y_trn, x_tst, y_tst):
#     ridge = Ridge(random_state=state, max_iter=10000, fit_intercept=True)
#     # parameters to use
#     solvers = ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga', 'lbfgs']
#     fitintercept = [True, False]
#     gparam_ridge_1 = {'solver': solvers,
#                       'fit_intercept': fitintercept, 
#                       'random_state': [state]}
#     # setting parameters
#     gs_ridge_1 = dcv.GridSearchCV(ridge, gparam_ridge_1, cv=n_folds, refit=True, n_jobs=-1,
#                                  scoring='neg_root_mean_squared_error',
#                                  return_train_score=True)
#     gs_ridge_1.fit(x_trn, y_trn)
#     ridge_model.test_rmse = rmse(gs_ridge_1, y_tst, x_tst)
    
def enet_model(x_trn, y_trn, x_tst, y_tst):
    enet = ElasticNet(max_iter=10000000, selection='random')
    # parameters to use
    alphas = [1e-4, 2.5e-4, 5e-4]
    l1ratio = [0.5, 0.7, 1]
    precomputes = [True, False]
    warmstart = [True, False]
    fitintercept = [True, False]
    gparam_enet_1 = {'alpha': alphas,
                     'l1_ratio': l1ratio,
                     'precompute': precomputes, 
                     'warm_start': warmstart,
                     'fit_intercept': fitintercept}
    # setting parameters
    gs_enet_1 = dcv.GridSearchCV(enet, gparam_enet_1, cv=n_folds, refit=True, n_jobs=-1,
                                scoring='neg_root_mean_squared_error', 
                                return_train_score=True)
    gs_enet_1.fit(x_trn, y_trn)
    enet_model.test_rmse = rmse(gs_enet_1, y_tst, x_tst)
    
def rfr_model(x_trn, y_trn, x_tst, y_tst):
    rfr = RandomForestRegressor()
    gparam_rfr = {}
    # setting parameters
    gs_rfr = dcv.GridSearchCV(rfr, gparam_rfr, cv=n_folds, refit=True, n_jobs=-1,
                             scoring='neg_root_mean_squared_error', return_train_score=True)
    gs_rfr.fit(x_trn, y_trn)
    rfr_model.test_rmse = rmse(gs_rfr, y_tst, x_tst)
    
def gbm_model(x_trn, y_trn, x_tst, y_tst):
    gbm = GradientBoostingRegressor()
    gparam_gbm = {}
    gs_gbm = dcv.GridSearchCV(gbm, gparam_gbm, cv=n_folds, refit=True, n_jobs=-1,
                          scoring='neg_root_mean_squared_error', return_train_score=True)
    gs_gbm.fit(x_trn, y_trn)
    gbm_model.test_rmse = rmse(gs_gbm, y_tst, x_tst)
    
def xgb_model(x_trn, y_trn, x_tst, y_tst):
    xgb_t = xgboost.XGBRegressor()
    gparam_xgb_t = {}
    gs_xgb_t = dcv.GridSearchCV(xgb_t, gparam_xgb_t, cv=n_folds, refit=True, n_jobs=-1,
                            scoring='neg_root_mean_squared_error', return_train_score=True)
    gs_xgb_t.fit(x_trn, y_trn)
    xgb_model.test_rmse = rmse(gs_xgb_t, y_tst, x_tst)

In [4]:
hp = copy.deepcopy(combo_nonedit)
state = 0
counter = 0

# using standard kfold split
n_folds = ms.KFold(n_splits=5, random_state=state, shuffle=True)

# lists of the functions separated by whether something is changed or added
stuff_add = [nbrhd_rank, totallivsf_add, bsmtfin_add, totalporchsf_add, 
             yrsremodtosold_add, fireplace_yes, pool_yes, totalbaths_add,
             totalbsmtbaths_add, conditions_edit]

# list of functions that will drop columns
stuff_drop = [drop_nbrhd, drop_grlivarea, drop_bsmtfinsf12,  drop_porches,
              drop_yrsoldremod, drop_fireplaces, drop_poolarea,
              drop_baths, drop_bsmtbaths]

# list of models to use
models = [mlr_model, lasso_model,  enet_model, rfr_model, gbm_model, xgb_model]

# using self-written function to generate permutations of 2 powersets
# looking at the max length
len(comb_list_generator(stuff_add, stuff_drop))

92378

In [5]:
def dum_scale(df):
    '''
    dummify and scale the input dataset
    '''
    # dummify the variables
    hp_dum = pd.get_dummies(df, drop_first=True)
    hp_dum_cols = list(hp_dum.columns)
    # scale the dataset
    for col in hp_dum_cols:
        hp_dum[col] = MinMaxScaler().fit_transform(np.array(hp_dum[col]).reshape(-1,1))
        hp_dum.columns = hp_dum_cols
    hp_dum_scale = copy.deepcopy(hp_dum)
    return hp_dum_scale

In [6]:
# def dum_scale(df):
#     # dummify the variables
#     hp_dum = cudf.get_dummies(df)
#     '''
#     since cuML's get_dummies does not yet support drop_first,
#     I wrote this as a part of the function that would drop 
#     the column with the highest count of 1s rather than dropping
#     the first (it was easier)
#     '''
#     obj_list = [string for string in hp_dum.columns if '_' in string]
#     cols_tracker = []
#     to_drop_list = []

#     for feat in obj_list:
#         # we are assuming that dummification has already occurred
#         # and that we are using the dummified hp_dum to figure out which
#         # columns to delete
#         feat_sp = str(feat.split('_')[0])
#         if feat_sp in cols_tracker:
#             continue
#         else:
#             # this gives us the original features that were dummified
#             cols_tracker.append(feat_sp)

#     for sub_cols in cols_tracker:
#         # reusing a dictionary to get the key of the highest value
#         tracker_sum = {}
#         cols = [col for col in hp_dum.columns if sub_cols in col]
#         # heatingqc can get accidentally inserted into this dictionary
#         if 'HeatingQC' in cols:
#             cols.remove('HeatingQC')
#         for col in cols:
#             # the value will be the sum of the column in order to find
#             # the max
#             tracker_sum[col] = hp_dum[col].sum()
#         to_drop = max(tracker_sum, key=tracker_sum.get)
#         # the max gets added to the list to drop
#         to_drop_list.append(to_drop)

#     hp_dum_dp = hp_dum.drop(to_drop_list, axis=1)
#     hp_dum_dp_cols = list(hp_dum_dp.columns)
    
#     for col in hp_dum_dp_cols:
#         hp_dum_dp[col] = MinMaxScaler().fit_transform(cp.array(hp_dum_dp[col]).reshape(-1,1))
#         hp_dum_dp.columns = hp_dum_dp_cols
#     dum_scale.hp_dum_dp = copy.deepcopy(hp_dum_dp)

In [7]:
# numbers will be indices later down
funcs_scores = {1: {}, 2: {}, 3: {}, 4: {}, 5: {}, 6: {}}

def apply_comb_list(com_list):
    counter = 0
    for cb_list in com_list:

        combo = copy.deepcopy(combo_nonedit)

        # converting any numbers to numeric type
        for col in combo.columns: 
            combo[col] = combo[col].astype('float', errors='ignore')

        # converting these 2 cols to string bc difficulties wi/ OneHotEncoding using Dask
        mo_yr = ['MoSold', 'YrSold']
        for my in mo_yr:
            if my in combo.columns:
                combo[[my]] = combo[[my]].astype(str)
        
        # converting the single string within the list to a list of strings
        cb_split = re.sub("['']", "", str(cb_list)).strip('][').split(', ')
        # converting the list of strings to a single string
        funcs_used = ', '.join(cb_split)
        
        # calling all of the functions within the list
        for cb in cb_split:
            eval(cb)(combo)
        
        # getting the train dataset for modeling after dummification
        hp_touse = dum_scale(combo).loc[hp_index]

        # setting up train and test sets
        xtrain, xtest, ytrain, ytest = ms.train_test_split(hp_touse, hp_logsp,
                                                           test_size=0.2, 
                                                           random_state=state)
        # Must flatten to fit
        ytrain = ytrain.values.flatten()

        # run the models
        for i, model in enumerate(models):
            model(xtrain, ytrain, xtest, ytest)
            # each model has a designated number (the order in the list: models)
            i+=1
            # add scores to the dictionary of dictionaries
            funcs_scores[i][funcs_used] = model.test_rmse
        
        # keeping track of rounds
        counter += 1
        print(f'Completed round {counter}')
    return funcs_scores

In [8]:
apply_comb_list(comb_list_generator(stuff_add, stuff_drop))

Completed round 1
Completed round 2
Completed round 3
Completed round 4
Completed round 5
Completed round 6
Completed round 7
Completed round 8
Completed round 9
Completed round 10
Completed round 11
Completed round 12
Completed round 13
Completed round 14
Completed round 15


KeyboardInterrupt: 

In [9]:
funcs_scores

{1: {'use_og_data': 0.18574218212882695,
  'nbrhd_rank, drop_nbrhd': 0.1989034017681056,
  'nbrhd_rank, drop_grlivarea': 0.11761582220842737,
  'nbrhd_rank, drop_bsmtfinsf12': 0.11776912795915088,
  'nbrhd_rank, drop_porches': 0.11496939504540309,
  'nbrhd_rank, drop_yrsoldremod': 0.1147755085650913,
  'nbrhd_rank, drop_fireplaces': 0.11629752848495616,
  'nbrhd_rank, drop_poolarea': 0.11685421536294467,
  'nbrhd_rank, drop_baths': 0.11503626856780419,
  'nbrhd_rank, drop_bsmtbaths': 0.11844458637250042,
  'totallivsf_add, drop_nbrhd': 0.1984026408863573,
  'totallivsf_add, drop_grlivarea': 0.18574218212883337,
  'totallivsf_add, drop_bsmtfinsf12': 0.18572547387919666,
  'totallivsf_add, drop_porches': 0.19706905853632073,
  'totallivsf_add, drop_yrsoldremod': 0.18567438582844265,
  'totallivsf_add, drop_fireplaces': 0.18832294514500764},
 2: {'use_og_data': 0.10547483199235509,
  'nbrhd_rank, drop_nbrhd': 0.1092793819349589,
  'nbrhd_rank, drop_grlivarea': 0.10568021462459619,
  'nbrh

In [10]:
for k_1 in funcs_scores:
    k_min = min(funcs_scores[k_1], key=funcs_scores[k_1].get)
    v_min = funcs_scores[k_1][k_min]
    print(k_1)
    print(k_min)
    print(v_min) 
    print('#' * 50)

1
nbrhd_rank, drop_yrsoldremod
0.1147755085650913
##################################################
2
nbrhd_rank, drop_baths
0.10386079849045537
##################################################
3
totallivsf_add, drop_grlivarea
0.1082115214450725
##################################################
4
nbrhd_rank, drop_porches
0.12022035834523405
##################################################
5
totallivsf_add, drop_porches
0.11196686607711986
##################################################
6
nbrhd_rank, drop_fireplaces
0.11935169074038192
##################################################
